In [ ]:
import os
import time
import json


# Install required libraries
! pip install pandas streamlit matplotlib seaborn psycopg2 python-dotenv requests sqlalchemy

# Import the libraries after installation
from dotenv import load_dotenv
import requests
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine

In [35]:
import requests
import pandas as pd
import os
import uuid  # For generating unique book IDs

# Your Google Books API key
API_KEY = os.getenv("API_KEY")

# Base URL for the Google Books API
BASE_URL = "https://www.googleapis.com/books/v1/volumes"

def fetch_books(query, max_entries=1000):
    """
    Fetch book data from Google Books API with pagination.

    :param query: Search query for books.
    :param max_entries: Maximum number of books to fetch.
    :return: List of book dictionaries.
    """
    if not API_KEY:
        raise ValueError("API_KEY is not set. Please set it in your environment variables.")

    all_books = []
    max_results = 40  # Maximum allowed by the API
    start_index = 0

    while len(all_books) < max_entries:
        params = {
            "q": query,
            "startIndex": start_index,
            "maxResults": max_results,
            "key": API_KEY
        }

        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()
        items = data.get("items", [])

        if not items:
            print("No more results available.")
            break

        for item in items:
            volume_info = item.get("volumeInfo", {})
            book = {
                "book_id": str(uuid.uuid4()),  # Unique identifier for each book
                "search_key": query,
                "book_title": volume_info.get("title", "No Title"),
                "book_subtitle": volume_info.get("subtitle", "N/A"),
                "book_authors": ", ".join(volume_info.get("authors", ["Unknown Author"])),
                "book_description": volume_info.get("description", "No Description"),
                "industryIdentifiers": ", ".join([identifier.get("identifier", "N/A") for identifier in volume_info.get("industryIdentifiers", [])]),
                "text_readingModes": volume_info.get("readingModes", {}).get("text", False),
                "image_readingModes": volume_info.get("readingModes", {}).get("image", False),
                "pageCount": volume_info.get("pageCount", "N/A"),
                "categories": ", ".join(volume_info.get("categories", ["N/A"])),
                "language": volume_info.get("language", "N/A"),
                "imageLinks": volume_info.get("imageLinks", {}).get("thumbnail", "No Image"),
                "ratingsCount": volume_info.get("ratingsCount", "N/A"),
                "averageRating": volume_info.get("averageRating", "N/A"),
                "country": volume_info.get("country", "N/A"),
                "saleability": volume_info.get("saleability", "N/A"),
                "isEbook": volume_info.get("isEbook", False),
                "amount_listPrice": volume_info.get("listPrice", {}).get("amount", "N/A"),
                "currencyCode_listPrice": volume_info.get("listPrice", {}).get("currencyCode", "N/A"),
                "amount_retailPrice": volume_info.get("retailPrice", {}).get("amount", "N/A"),
                "currencyCode_retailPrice": volume_info.get("retailPrice", {}).get("currencyCode", "N/A"),
                "buyLink": volume_info.get("infoLink", "No Buy Link"),
                "year": volume_info.get("publishedDate", "N/A").split("-")[0],  # Extract year
                "publisher": volume_info.get("publisher", "N/A")
            }
            all_books.append(book)

        # Increment the start index
        start_index += max_results
        print(f"Fetched {len(all_books)} books so far...")

        # Stop if we've hit the max_entries limit
        if len(all_books) >= max_entries:
            break

    return all_books[:max_entries]

def save_books_to_csv(books, filename="BooksData.csv"):
    """
    Save book data to a CSV file.

    :param books: List of book dictionaries.
    :param filename: Name of the CSV file to save.
    """
    df = pd.DataFrame(books)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Main function to fetch and save books
def main():
    query = "fiction"  # Change to your preferred query
    max_entries = 1000
    try:
        books = fetch_books(query, max_entries)
        save_books_to_csv(books)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Fetched 40 books so far...
Fetched 80 books so far...
Fetched 120 books so far...
Fetched 160 books so far...
Fetched 200 books so far...
Fetched 240 books so far...
Fetched 280 books so far...
Fetched 320 books so far...
Fetched 360 books so far...
Fetched 400 books so far...
Fetched 440 books so far...
Fetched 480 books so far...
Fetched 520 books so far...
Fetched 560 books so far...
Fetched 600 books so far...
Fetched 640 books so far...
Fetched 680 books so far...
Fetched 720 books so far...
Fetched 760 books so far...
Fetched 800 books so far...
Fetched 840 books so far...
Fetched 880 books so far...
Fetched 920 books so far...
Fetched 960 books so far...
Fetched 1000 books so far...
Data saved to BooksData.csv


In [36]:
# Read the CSV file into a DataFrame
df = pd.read_csv('BooksData.csv')
#remove duplicates in the csv
df.drop_duplicates()
# Convert NaN into 0 to avoid error while converting columns into integer
df['pageCount'] = df['pageCount'].fillna(0).astype(int)
df['ratingsCount'] = df['ratingsCount'].fillna(0).astype(int)


# Change data types of the columns
df['book_id'] = df['book_id'].astype(str)
df['search_key'] = df['search_key'].astype(str)
df['book_title'] = df['book_title'].astype(str)
df['book_subtitle'] = df['book_subtitle'].astype(str)
df['book_authors'] = df['book_authors'].astype(str)
df['book_description'] = df['book_description'].astype(str)
df['industryIdentifiers'] = df['industryIdentifiers'].astype(str)
df['text_readingModes'] = df['text_readingModes'].astype(bool)
df['image_readingModes'] = df['image_readingModes'].astype(bool)
df['pageCount'] = df['pageCount'].astype(int)
df['categories'] = df['categories'].astype(str)
df['language'] = df['language'].astype(str)
df['imageLinks'] = df['imageLinks'].astype(str)
df['ratingsCount'] = df['ratingsCount'].astype(int)
df['averageRating'] = df['averageRating'].astype(float)
df['country'] = df['country'].astype(str)
df['saleability'] = df['saleability'].astype(str)
df['isEbook'] = df['isEbook'].astype(bool)
df['amount_listPrice'] = df['amount_listPrice'].astype(float)
df['currencyCode_listPrice'] = df['currencyCode_listPrice'].astype(str)
df['amount_retailPrice'] = df['amount_retailPrice'].astype(float)
df['currencyCode_retailPrice'] = df['currencyCode_retailPrice'].astype(str)
df['buyLink'] = df['buyLink'].astype(str)
df['year'] = df['year'].astype(str)
df['publisher'] = df['publisher'].astype(str)

# Check the data types after conversion
#print(df.dtypes)


book_id                      object
search_key                   object
book_title                   object
book_subtitle                object
book_authors                 object
book_description             object
industryIdentifiers          object
text_readingModes              bool
image_readingModes             bool
pageCount                     int64
categories                   object
language                     object
imageLinks                   object
ratingsCount                  int64
averageRating               float64
country                      object
saleability                  object
isEbook                        bool
amount_listPrice            float64
currencyCode_listPrice       object
amount_retailPrice          float64
currencyCode_retailPrice     object
buyLink                      object
year                         object
publisher                    object
dtype: object


In [37]:
# Load data into SQL
# Load environment variables from the .env file txt 
# Database connection settings from environment variables
load_dotenv()

host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
database = os.getenv("DB_NAME")
password = os.getenv("DB_PASSWORD")


engine = create_engine(f"postgresql://{user}:{password}@{host}/{database}")

try:
    df.to_sql('books', engine, if_exists='append', index=False)
    print("Successfully inserted")
except Exception as e:
    print(f"Error inserting data: {e}")


Successfully inserted
